In [1]:
%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
%pip install torchsummary

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import os

from IPython.display import clear_output

from pathlib import Path

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm

import random

import transformers
from transformers import BertModel
from transformers.models.bert.tokenization_bert import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

from torch.utils.tensorboard import SummaryWriter

In [3]:
def fix_seed(seed: int = 42) -> None:
    """
    Set seeds for reproducibility
    :param seed: seed value
    """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [4]:
fix_seed(42)

In [5]:
model_cache_path = Path('./model_cache')
model_cache_path.mkdir(parents=True, exist_ok=True)

base_model_name = "cointegrated/rubert-tiny"

In [6]:
trained_model_path = Path('./trained_model_artifacts')
trained_model_path.mkdir(parents=True, exist_ok=True)

In [9]:
# train_df = pd.read_parquet('../../mnt/s3/hsedatafitpredict1392/mega_train.parquet')
train_df = pd.read_csv('../../mnt/s3/hsedatafitpredict1392/pca_df_test.csv')

In [12]:
train_df

,sentence,communication,price,quality,safety,not_def,sentiment,y
0,10.12.2020 20:57 Хотелось бы выразить огромну...,0.0,0.0,3.0,0.0,0.0,1.000000,1
1,"!, на что сотрудник банка ответила мне что дан...",1.0,0.0,0.0,0.0,2.0,-0.666667,-1
2,"!, тем самым оставив меня без средств к сущест...",0.0,0.0,1.0,0.0,2.0,-1.000000,-1
3,!16 ноября сего года я обращаюсь к администрат...,0.0,0.0,0.0,0.0,3.0,-0.333333,0
4,!22.02.2020 заблокировали счет якобы из-за про...,3.0,0.0,0.0,0.0,0.0,-1.000000,-1
...,...,...,...,...,...,...,...,...
7161,я своевременно и в полном объеме исполнял ВСЕ ...,0.0,0.0,0.0,0.0,2.0,0.000000,0
7162,я у сотрудника спросила- получила ответ. но со...,3.0,0.0,0.0,1.0,0.0,-1.000000,-1
7163,"– восхищаюсь я. Проходит еще неделя... Ну, я д...",2.0,0.0,0.0,0.0,1.0,0.000000,0
7164,"…Вы своих клиентов совсем не уважаете, считая ...",2.0,1.0,1.0,0.0,0.0,-1.000000,-1


In [13]:
tokenizer = transformers.BertTokenizer.from_pretrained(base_model_name, cache_dir = model_cache_path)
base_rubert = BertModel.from_pretrained(base_model_name, cache_dir = model_cache_path)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [16]:
embeddings = {}

In [37]:
embeddings = {}

embed_bert_cls(train_df.sentence.iloc[0], base_rubert, tokenizer)==train_emb_df.iloc[0, 8:]

0      True
1      True
2      True
3      True
4      True
       ... 
307    True
308    True
309    True
310    True
311    True
Name: 0, Length: 312, dtype: bool

In [18]:
for count, sentence in enumerate(train_df.sentence):
    embeddings[count] = embed_bert_cls(sentence, base_rubert, tokenizer)
emb_df = pd.DataFrame(embeddings).T
train_emb_df = pd.concat([train_df, emb_df], axis = 1)
train_emb_df.columns = [str(i) for i in train_emb_df.columns]

In [32]:
train_emb_df

,sentence,communication,price,quality,safety,not_def,sentiment,y,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,...,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311
0,10.12.2020 20:57 Хотелось бы выразить огромну...,0.0,0.0,3.0,0.0,0.0,1.000000,1,0.095926,0.027878,-0.058136,-0.016623,0.027831,0.076080,-0.028747,-0.018289,-0.003320,-0.002076,0.037846,-0.023627,0.036656,0.048770,0.030455,0.006952,0.043346,-0.050525,0.033863,0.029311,0.078209,-0.026999,0.012212,0.031110,0.056866,0.007879,0.026826,0.005439,-0.021732,-0.031723,0.041375,0.014019,...,-0.070696,0.040503,0.091467,-0.022976,0.006216,-0.037349,-0.015663,-0.042905,0.096232,0.078046,-0.018239,0.104680,0.106103,0.031331,0.088598,-0.033268,0.007504,0.095698,0.065530,0.030994,0.024570,-0.007427,-0.051598,0.009558,0.027560,0.003978,0.015847,-0.006044,-0.007663,-0.029089,0.039345,-0.058049,-0.021589,0.047853,0.008267,-0.022683,0.048195,-0.040051,0.060394,-0.087500
1,"!, на что сотрудник банка ответила мне что дан...",1.0,0.0,0.0,0.0,2.0,-0.666667,-1,-0.048135,-0.042267,-0.086132,-0.014711,-0.041393,0.001635,-0.026059,-0.023906,0.016925,0.004409,0.030816,0.014250,0.061080,0.047116,-0.054787,-0.073977,0.022510,-0.026220,0.024860,0.050068,0.023539,0.007542,0.025501,0.057262,0.153315,0.016074,-0.018198,0.039547,-0.017512,0.010320,0.041569,0.041473,...,-0.074579,0.036425,-0.010796,-0.004769,-0.025803,-0.007568,0.046609,-0.020262,0.033571,0.023054,-0.074288,0.025581,0.075271,0.013288,-0.070866,0.037449,0.024293,-0.008147,-0.021381,0.037102,-0.066186,0.034778,-0.021497,-0.056985,-0.015758,-0.035495,-0.026285,0.014208,0.062482,0.041706,0.047161,-0.026783,-0.009019,0.023077,-0.018781,-0.020640,0.072834,-0.077632,0.038530,-0.149410
2,"!, тем самым оставив меня без средств к сущест...",0.0,0.0,1.0,0.0,2.0,-1.000000,-1,0.037359,0.028186,-0.063541,-0.094592,0.017141,0.028424,0.017555,-0.058833,-0.032220,0.018059,0.059904,0.017018,0.011426,-0.001337,0.019992,0.014264,0.019165,-0.023469,0.063555,0.104933,0.047445,0.048078,-0.027219,-0.020699,0.071718,-0.048365,-0.016438,0.007557,0.024456,0.071446,-0.034367,-0.062093,...,-0.106949,0.045543,0.022683,0.001252,0.041202,-0.066694,0.050656,0.043666,0.065812,-0.056879,-0.075688,0.031824,-0.009050,-0.013238,0.000955,0.114893,0.022653,-0.024259,-0.000068,0.046412,-0.080042,0.031940,-0.044985,0.039570,0.018766,0.033080,-0.005886,-0.004528,-0.023042,-0.008942,0.079089,-0.041286,-0.002672,0.039177,0.062531,-0.013165,0.071845,-0.033408,0.080519,-0.047402
3,!16 ноября сего года я обращаюсь к администрат...,0.0,0.0,0.0,0.0,3.0,-0.333333,0,0.088608,-0.028121,-0.050769,-0.047755,0.026834,0.028098,-0.004484,-0.067947,0.045673,0.015599,0.033622,0.027069,0.046763,0.104692,-0.025431,0.006833,0.058766,-0.063744,0.019495,0.041909,0.037717,-0.023874,-0.058842,0.001774,0.040827,-0.039593,-0.030874,-0.053938,-0.075533,0.029989,-0.050798,0.069755,...,-0.106801,0.016381,0.041700,0.043425,0.009434,-0.084782,-0.001008,0.022590,0.041143,0.039203,-0.080154,0.009494,0.026451,0.027974,-0.040576,0.041519,-0.057238,0.037700,-0.025383,0.030307,-0.012037,0.046361,-0.095273,0.022201,0.033705,0.022263,-0.020702,-0.006572,0.025646,-0.047426,0.033524,-0.054217,0.014685,0.010346,0.073833,-0.076165,0.038723,0.039786,0.052765,-0.049013
4,!22.02.2020 заблокировали счет якобы из-за про...,3.0,0.0,0.0,0.0,0.0,-1.000000,-1,0.074634,-0.038165,-0.051844,-0.015446,-0.005628,-0.002990,-0.060078,-0.098032,0.068792,0.001815,0.083190,-0.040806,0.034463,0.050555,-0.045979,-0.022340,0.000438,-0.036496,0.048625,-0.041465,0.035430,-0.045134,-0.005481,0.071890,0.075424,-0.007514,0.015820,-0.001924,-0.019186,0.024450,0.009244,0.029423,...,-0.042638,-0.010912,0.047654,0.014334,0.011989,-0.015173,-0.022972,0.016561,0.111511,0.024446,-0.009501,0.012639,0.041016,-0.017982,0.069481,0.006222,0.066721,0.043587,0.020502,0.003668,0.024744,-0.003102,0.013972,0.00016

In [33]:
train_emb_df.to_parquet('../../mnt/s3/hsedatafitpredict1392/catboost/mega_train_embeddings_bert.parquet')

In [42]:
test_df = pd.read_csv('../../mnt/s3/hsedatafitpredict1392/new_test.csv', index_col=0)

In [46]:
test_df

,texts
4036,15.03.2022 обратился на горячую линию для закр...
5804,"Уже который год в ТКБ не решается ""глобальная ..."
2752,Добрый день. Хочу оставить отзыв о пользовании...
1921,"Добрый день Сегодня, зайдя в свой личный кабин..."
7374,"Обслуживаюсь в Тинькофф пару лет, возникла жес..."
...,...
146,Отвратительный сервис и отношение к клиентам! ...
2677,28.04.2022 обратилась в банк о возможности пер...
4481,В начале 2021 года была акция по выплате 8% ке...
4112,Бездействие банка и некомпетентность сотрудник...


In [47]:
embeddings = {}
for count, sentence in enumerate(test_df.texts):
    embeddings[count] = embed_bert_cls(sentence, base_rubert, tokenizer)
emb_df = pd.DataFrame(embeddings).T
test_emb_df = pd.concat([test_df, emb_df], axis = 1)
test_emb_df.columns = [str(i) for i in test_emb_df.columns]

In [55]:
test_df.reset_index(inplace=True)

In [57]:
test_df.drop(['index'], axis=1, inplace=True)

In [58]:
test_emb_df = pd.concat([test_df, emb_df], axis = 1)
test_emb_df.columns = [str(i) for i in test_emb_df.columns]

In [59]:
test_emb_df

,texts,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311
0,15.03.2022 обратился на горячую линию для закр...,0.036882,-0.041032,-0.059115,0.041549,0.000174,0.013476,-0.034519,-0.086633,0.083429,0.025479,0.046240,-0.025328,0.039003,0.073136,-0.063955,-0.036371,0.012510,0.008714,0.067711,-0.064029,0.049956,-0.063688,-0.015649,0.081211,0.099646,0.036903,0.018191,-0.025869,-0.005230,-0.006423,0.007809,-0.003234,0.013556,0.047096,0.026831,-0.054476,0.019187,-0.017453,0.092827,...,-0.018768,0.038388,0.053456,-0.018858,0.012114,-0.005442,-0.032532,-0.062207,0.122246,0.060579,-0.029866,0.056710,0.081131,0.024721,0.053481,-0.017727,0.063376,0.090902,-0.009570,0.025619,0.062573,-0.001860,-0.017456,-0.025372,-0.016689,-0.002079,-0.005652,-0.001593,0.039915,-0.057905,0.058940,-0.069663,-0.016080,0.066492,-0.014808,-0.050818,0.064962,-0.064996,0.042211,-0.009411
1,"Уже который год в ТКБ не решается ""глобальная ...",0.011745,0.043900,-0.045469,0.027484,-0.000249,0.022167,-0.043259,-0.107886,0.082646,0.001381,-0.005283,0.007836,0.019240,0.089958,0.002694,-0.029447,0.044546,-0.013794,0.021677,-0.052669,0.061383,-0.035149,-0.030059,0.047558,0.096897,0.021212,0.027431,-0.047664,-0.026788,0.028034,-0.057237,0.001579,0.009083,0.018026,-0.038252,-0.035163,-0.003470,-0.013148,0.127647,...,0.013283,0.007088,0.059108,0.044500,0.005217,-0.021445,0.007809,-0.014718,0.087687,0.047679,-0.054716,-0.004214,0.095156,0.052201,0.018018,0.038127,-0.005495,0.102023,-0.011222,0.033275,0.005551,-0.022180,-0.007919,0.003433,0.014172,0.026880,-0.008015,-0.035823,0.002496,0.020102,0.003268,-0.045319,-0.025314,0.060521,-0.007234,0.005245,0.044774,-0.019223,0.034248,-0.000405
2,Добрый день. Хочу оставить отзыв о пользовании...,0.052357,0.048711,-0.049086,0.010275,0.016346,-0.007538,-0.049288,-0.023505,0.001033,-0.025347,0.015953,-0.038844,0.041811,0.044048,-0.021567,-0.052934,0.046452,-0.015045,0.027772,0.012242,0.039769,-0.030317,0.019555,0.070055,0.115316,0.017464,0.036046,0.000714,-0.009243,-0.010224,-0.011036,-0.009175,0.006026,-0.013645,-0.017692,-0.019563,-0.003571,0.033163,0.080787,...,-0.009454,0.026358,0.026417,0.007089,0.013427,-0.033938,-0.030580,-0.039779,0.088973,0.082657,-0.084850,0.058767,0.098634,0.063269,0.063630,0.039216,-0.012119,0.044927,0.034138,0.035100,0.023224,-0.043766,-0.051332,0.026822,0.043360,0.036258,-0.022550,-0.006555,0.022693,0.029109,0.012900,-0.017019,0.009260,0.081971,-0.019074,-0.013111,0.025934,-0.052603,0.013478,-0.106095
3,"Добрый день Сегодня, зайдя в свой личный кабин...",0.040339,0.067312,-0.026479,0.016007,-0.026469,-0.007504,-0.022122,-0.090189,0.031208,-0.009807,0.066894,0.005961,0.038576,0.086348,-0.036788,-0.025501,-0.009671,-0.053752,0.041505,-0.027932,0.099761,0.026816,-0.020752,0.079228,0.096063,-0.003367,0.003788,-0.014459,-0.019166,0.075000,0.039794,-0.011724,-0.006403,-0.013283,0.009450,-0.062904,0.047818,0.006582,0.056481,...,-0.041259,0.027421,-0.008311,0.052483,-0.010251,-0.015186,0.000994,0.024095,0.102665,-0.036231,-0.041231,0.052047,0.064200,0.048588,0.002194,0.038303,-0.028692,0.043663,-0.011206,-0.001092,-0.062588,-0.021831,-0.022565,0.048528,0.006034,0.002181,-0.081727,-0.030483,0.029448,0.002978,-0.026050,-0.088938,0.037771,0.047856,0.011872,-0.004560,0.012706,-0.060824,0.018362,-0.054970
4,"Обслуживаюсь в Тинькофф пару лет, возникла жес...",0.067984,-0.052932,-0.043885,-0.030161,-0.010461,-0.011198,-0.050009,-0.106517,0.027698,0.003380,0.064194,-0.027480,0.016363,0.036914,-0.033764,-0.015171,0.017338,-0.013869,0.025403,0.001079,0.053512,-0.028796,-0.000946,0.038098,0.130368,0.034909,0.066224,0.029026,0.056891,-0.034188,-0.016706,-0.007549,0.018943,0.019953,0.016754,0.006187,0.044065,0.019409,0.080692,...,0.000899,0.009507,0.004457,0.023632,-0.000308,0.008231,0.008127,0.018927,0.046995,0

In [62]:
test_emb_df.to_parquet('../../mnt/s3/hsedatafitpredict1392/catboost/embedding_test.parquet')

In [61]:
#!g1.1
test_emb_df

,texts,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311
0,15.03.2022 обратился на горячую линию для закр...,0.036882,-0.041032,-0.059115,0.041549,0.000174,0.013476,-0.034519,-0.086633,0.083429,0.025479,0.046240,-0.025328,0.039003,0.073136,-0.063955,-0.036371,0.012510,0.008714,0.067711,-0.064029,0.049956,-0.063688,-0.015649,0.081211,0.099646,0.036903,0.018191,-0.025869,-0.005230,-0.006423,0.007809,-0.003234,0.013556,0.047096,0.026831,-0.054476,0.019187,-0.017453,0.092827,...,-0.018768,0.038388,0.053456,-0.018858,0.012114,-0.005442,-0.032532,-0.062207,0.122246,0.060579,-0.029866,0.056710,0.081131,0.024721,0.053481,-0.017727,0.063376,0.090902,-0.009570,0.025619,0.062573,-0.001860,-0.017456,-0.025372,-0.016689,-0.002079,-0.005652,-0.001593,0.039915,-0.057905,0.058940,-0.069663,-0.016080,0.066492,-0.014808,-0.050818,0.064962,-0.064996,0.042211,-0.009411
1,"Уже который год в ТКБ не решается ""глобальная ...",0.011745,0.043900,-0.045469,0.027484,-0.000249,0.022167,-0.043259,-0.107886,0.082646,0.001381,-0.005283,0.007836,0.019240,0.089958,0.002694,-0.029447,0.044546,-0.013794,0.021677,-0.052669,0.061383,-0.035149,-0.030059,0.047558,0.096897,0.021212,0.027431,-0.047664,-0.026788,0.028034,-0.057237,0.001579,0.009083,0.018026,-0.038252,-0.035163,-0.003470,-0.013148,0.127647,...,0.013283,0.007088,0.059108,0.044500,0.005217,-0.021445,0.007809,-0.014718,0.087687,0.047679,-0.054716,-0.004214,0.095156,0.052201,0.018018,0.038127,-0.005495,0.102023,-0.011222,0.033275,0.005551,-0.022180,-0.007919,0.003433,0.014172,0.026880,-0.008015,-0.035823,0.002496,0.020102,0.003268,-0.045319,-0.025314,0.060521,-0.007234,0.005245,0.044774,-0.019223,0.034248,-0.000405
2,Добрый день. Хочу оставить отзыв о пользовании...,0.052357,0.048711,-0.049086,0.010275,0.016346,-0.007538,-0.049288,-0.023505,0.001033,-0.025347,0.015953,-0.038844,0.041811,0.044048,-0.021567,-0.052934,0.046452,-0.015045,0.027772,0.012242,0.039769,-0.030317,0.019555,0.070055,0.115316,0.017464,0.036046,0.000714,-0.009243,-0.010224,-0.011036,-0.009175,0.006026,-0.013645,-0.017692,-0.019563,-0.003571,0.033163,0.080787,...,-0.009454,0.026358,0.026417,0.007089,0.013427,-0.033938,-0.030580,-0.039779,0.088973,0.082657,-0.084850,0.058767,0.098634,0.063269,0.063630,0.039216,-0.012119,0.044927,0.034138,0.035100,0.023224,-0.043766,-0.051332,0.026822,0.043360,0.036258,-0.022550,-0.006555,0.022693,0.029109,0.012900,-0.017019,0.009260,0.081971,-0.019074,-0.013111,0.025934,-0.052603,0.013478,-0.106095
3,"Добрый день Сегодня, зайдя в свой личный кабин...",0.040339,0.067312,-0.026479,0.016007,-0.026469,-0.007504,-0.022122,-0.090189,0.031208,-0.009807,0.066894,0.005961,0.038576,0.086348,-0.036788,-0.025501,-0.009671,-0.053752,0.041505,-0.027932,0.099761,0.026816,-0.020752,0.079228,0.096063,-0.003367,0.003788,-0.014459,-0.019166,0.075000,0.039794,-0.011724,-0.006403,-0.013283,0.009450,-0.062904,0.047818,0.006582,0.056481,...,-0.041259,0.027421,-0.008311,0.052483,-0.010251,-0.015186,0.000994,0.024095,0.102665,-0.036231,-0.041231,0.052047,0.064200,0.048588,0.002194,0.038303,-0.028692,0.043663,-0.011206,-0.001092,-0.062588,-0.021831,-0.022565,0.048528,0.006034,0.002181,-0.081727,-0.030483,0.029448,0.002978,-0.026050,-0.088938,0.037771,0.047856,0.011872,-0.004560,0.012706,-0.060824,0.018362,-0.054970
4,"Обслуживаюсь в Тинькофф пару лет, возникла жес...",0.067984,-0.052932,-0.043885,-0.030161,-0.010461,-0.011198,-0.050009,-0.106517,0.027698,0.003380,0.064194,-0.027480,0.016363,0.036914,-0.033764,-0.015171,0.017338,-0.013869,0.025403,0.001079,0.053512,-0.028796,-0.000946,0.038098,0.130368,0.034909,0.066224,0.029026,0.056891,-0.034188,-0.016706,-0.007549,0.018943,0.019953,0.016754,0.006187,0.044065,0.019409,0.080692,...,0.000899,0.009507,0.004457,0.023632,-0.000308,0.008231,0.008127,0.018927,0.046995,0

In [ ]:
#!g1.1
